Quinto ejercicio de Spark Structured Streaming. Lista todos datos consumidos desde un topic de kafka cuyos datos son enviados en formato JSON como por ejemplo:
{"nombre":"JOSE ANTONIO","edad":38,"peso":75.5,"direccion":"C/ DEL LIMONERO 39, PISO 1-A"}

In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

In [2]:
spark = SparkSession\
        .builder\
        .appName("StructuredKafkaJSON")\
        .getOrCreate()

In [3]:
# Define schema de json
schema = StructType() \
        .add("nombre", StringType()) \
        .add("apellido", StringType())\
        .add("edad", IntegerType()) \
        .add("peso", FloatType()) \
        .add("sexo", StringType())\
        .add("direccion", StringType())

In [4]:
# Create DataSet representing the stream of input lines from kafka
# Es necesario de antemano haber creado el topic llamado wordcount_topic
#
lines = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "json2_topic")\
        .load()\
        .selectExpr("value")\
        .select(from_json(col("value"), schema).alias("parsed_value"))\
        .select("parsed_value.nombre", "parsed_value.apellido", "parsed_value.edad")

AnalysisException: "cannot resolve 'jsontostructs(`value`)' due to data type mismatch: argument 1 requires string type, however, '`value`' is of binary type.;;\n'Project [jsontostructs(StructField(nombre,StringType,true), StructField(apellido,StringType,true), StructField(edad,IntegerType,true), StructField(peso,FloatType,true), StructField(sexo,StringType,true), StructField(direccion,StringType,true), value#8, Some(Europe/Madrid), true) AS parsed_value#22]\n+- Project [value#8]\n   +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@4999c14e, kafka, Map(subscribe -> json2_topic, kafka.bootstrap.servers -> localhost:9092), [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@7df0f4df,kafka,List(),None,List(),None,Map(subscribe -> json2_topic, kafka.bootstrap.servers -> localhost:9092),None), kafka, [key#0, value#1, topic#2, partition#3, offset#4L, timestamp#5, timestampTy

In [ ]:
# Una vez iniciado el procesamiento del flujo empezar a insertar elementos al topic de kafka
# Ver resultados en la consola donde se ejecuto el jupyter notebook
query = lines.writeStream \
        .outputMode("append") \
        .format("console") \
        .start()

query.awaitTermination()